# NYC taxi data regression

**Requirements** - In order to benefit from this tutorial, you will need:
- A basic understanding of Machine Learning
- An Azure account with an active subscription - [Create an account for free](https://azure.microsoft.com/free/?WT.mc_id=A261C142F)
- An Azure ML workspace with computer cluster - [Configure workspace](../../configuration.ipynb)
- A python environment
- Installed Azure Machine Learning Python SDK v2 - [install instructions](../../../README.md) - check the getting started section

**Learning Objectives** - By the end of this tutorial, you should be able to:
- Connect to your AML workspace from the Python SDK
- Define different `CommandComponent` using YAML
- Create `Pipeline` load these components from YAML

**Motivations** - This notebook explains how to load component via SDK then use these components to build pipeline. We use NYC dataset, build pipeline with five steps, prep data, transform data, train model, predict results and evaluate model performance.

# 1. Connect to Azure Machine Learning Workspace

The [workspace](https://docs.microsoft.com/en-us/azure/machine-learning/concept-workspace) is the top-level resource for Azure Machine Learning, providing a centralized place to work with all the artifacts you create when you use Azure Machine Learning. In this section we will connect to the workspace in which the job will be run.

## 1.1 Import the required libraries

In [12]:
# import required libraries
from azure.identity import DefaultAzureCredential, InteractiveBrowserCredential

from azure.ai.ml import MLClient, Input, Output
from azure.ai.ml.dsl import pipeline
from azure.ai.ml import load_component

## 1.2 Configure credential

We are using `DefaultAzureCredential` to get access to workspace. 
`DefaultAzureCredential` should be capable of handling most Azure SDK authentication scenarios. 

Reference for more available credentials if it does not work for you: [configure credential example](../../configuration.ipynb), [azure-identity reference doc](https://docs.microsoft.com/en-us/python/api/azure-identity/azure.identity?view=azure-python).

In [2]:
try:
    credential = DefaultAzureCredential()
    # Check if given credential can get token successfully.
    credential.get_token("https://management.azure.com/.default")
except Exception as ex:
    # Fall back to InteractiveBrowserCredential in case DefaultAzureCredential not work
    credential = InteractiveBrowserCredential()

## 1.3 Get a handle to the workspace

We use config file to connect to a workspace. The Azure ML workspace should be configured with computer cluster. [Check this notebook for configure a workspace](../../configuration.ipynb)

In [3]:
# Get a handle to workspace
ml_client = MLClient.from_config(credential=credential)

# Retrieve an already attached Azure Machine Learning Compute.
cluster_name = "cpu-cluster"
print(ml_client.compute.get(cluster_name))

Found the config file in: c:\Users\sapa\OneDrive - Microsoft\FTA_Projects\AzureMLPlayGround\config.json


AmlCompute({'type': 'amlcompute', 'created_on': None, 'provisioning_state': 'Succeeded', 'provisioning_errors': None, 'name': 'cpu-cluster', 'description': None, 'tags': {}, 'properties': {}, 'id': '/subscriptions/89c37dd8-94bb-4870-98e0-1cfb98c0262e/resourceGroups/rg-mlopsv2-sapa02dev/providers/Microsoft.MachineLearningServices/workspaces/mlw-mlopsv2-sapa02dev/computes/cpu-cluster', 'Resource__source_path': None, 'base_path': 'c:\\Users\\sapa\\OneDrive - Microsoft\\FTA_Projects\\AzureMLPlayGround\\sdkv2\\2c_nyc_taxi_data_regression', 'creation_context': None, 'serialize': <msrest.serialization.Serializer object at 0x000001C614744DC8>, 'resource_id': None, 'location': 'westus', 'size': 'STANDARD_DS3_V2', 'min_instances': 0, 'max_instances': 4, 'idle_time_before_scale_down': 120.0, 'identity': None, 'ssh_public_access_enabled': True, 'ssh_settings': None, 'network_settings': None, 'tier': 'dedicated'})


# 2. Build pipeline

In [31]:
parent_dir = ""

# 1. Load components
prepare_data = load_component(path=parent_dir + "./prep.yml")
transform_data = load_component(path=parent_dir + "./transform.yml")
train_model = load_component(path=parent_dir + "./train.yml")
predict_result = load_component(path=parent_dir + "./predict.yml")
score_data = load_component(path=parent_dir + "./score.yml")

# 2. Construct pipeline
@pipeline()
def nyc_taxi_data_regression(pipeline_job_input):
    """NYC taxi data regression example."""
    prepare_sample_data = prepare_data(raw_data=pipeline_job_input)
    transform_sample_data = transform_data(
        clean_data=prepare_sample_data.outputs.prep_data
    )
    train_with_sample_data = train_model(
        training_data=transform_sample_data.outputs.transformed_data
    )
    predict_with_sample_data = predict_result(
        model_input=train_with_sample_data.outputs.model_output,
        test_data=train_with_sample_data.outputs.test_data,
    )
    score_with_sample_data = score_data(
        predictions=predict_with_sample_data.outputs.predictions,
        model=train_with_sample_data.outputs.model_output,
    )
    return {
        "pipeline_job_prepped_data": prepare_sample_data.outputs.prep_data
        ,
        "pipeline_job_transformed_data": transform_sample_data.outputs.transformed_data,
        "pipeline_job_trained_model": train_with_sample_data.outputs.model_output,
        "pipeline_job_test_data": train_with_sample_data.outputs.test_data,
        "pipeline_job_predictions": predict_with_sample_data.outputs.predictions,
        "pipeline_job_score_report": score_with_sample_data.outputs.score_report,
    }


pipeline_job = nyc_taxi_data_regression(
    Input(type="uri_folder", path="azureml://datastores/blob_example/paths/nyctaxiexample")
    
)
# demo how to change pipeline output settings
# pipeline_job.outputs.pipeline_job_prepped_data.mode = "rw_mount"
pipeline_job.outputs.pipeline_job_prepped_data = Output(type="uri_folder", path="azureml://datastores/blob_example/paths/nyctaxiexample/output1")
# pipeline_job.outputs.pipeline_job_prepped_data.type = "uri_folder"



# set pipeline level compute
pipeline_job.settings.default_compute = "cpu-cluster"
# set pipeline level datastore
pipeline_job.settings.default_datastore = "workspaceblobstore"

## 3. Submit pipeline job

In [32]:
# submit job to workspace
pipeline_job = ml_client.jobs.create_or_update(
    pipeline_job, experiment_name="pipeline_samples"
)
pipeline_job

Experiment,Name,Type,Status,Details Page
pipeline_samples,jolly_pea_8yshgvxg0m,pipeline,Preparing,Link to Azure Machine Learning studio


In [10]:
# Wait until the job completes
ml_client.jobs.stream(pipeline_job.name)

RunId: blue_loquat_g4f1zzdbyj
Web View: https://ml.azure.com/runs/blue_loquat_g4f1zzdbyj?wsid=/subscriptions/89c37dd8-94bb-4870-98e0-1cfb98c0262e/resourcegroups/rg-mlopsv2-sapa02dev/workspaces/mlw-mlopsv2-sapa02dev

Streaming logs/azureml/executionlogs.txt

[2022-09-20 20:33:55Z] Submitting 1 runs, first five are: d3fe60e7:7fdc19eb-1437-404d-8c50-d6f4aa7e7783
[2022-09-20 20:36:11Z] Completing processing run id 7fdc19eb-1437-404d-8c50-d6f4aa7e7783.
[2022-09-20 20:36:11Z] Submitting 1 runs, first five are: 0e9d58ca:f4edf9db-ea3d-4a41-a47f-697e854be6e5
[2022-09-20 20:36:31Z] Completing processing run id f4edf9db-ea3d-4a41-a47f-697e854be6e5.
[2022-09-20 20:36:31Z] Submitting 1 runs, first five are: f9bd7074:5573bf30-7525-44dc-a5fe-b3cad1df250c
[2022-09-20 20:37:01Z] Completing processing run id 5573bf30-7525-44dc-a5fe-b3cad1df250c.
[2022-09-20 20:37:02Z] Submitting 1 runs, first five are: 733e2cb9:c25397f6-1885-4ae5-b28c-46b838733f61
[2022-09-20 20:37:26Z] Completing processing run id c253

# Next Steps
You can see further examples of running a pipeline job [here](../)